# Battle of the Pizzas

## Introduction

Pizza has always been favorite dish in the United States and there's no doubt about it. However, thw question "which city makes the best pizza?" has always been on going debate. The problem we aim to solve is to analyze the most popular Pizza cities and their pizzerias offer the best pizzas. Our target audience are international pizza lovers.

## Data

We will use the FourSquare API to collect data about locations of Pizza stores in 5 major US cities which are: New York, NY, Chicago, IL, New Haven, CT, San Francisco, CA and Orlando, FL. These are one of the most popular pizza  cities. The data I will need will include:
-Location (long/lat)
-Review/rating

## Methodology

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import bs4 as bs
import urllib.request

print('Libraries imported.')

Libraries imported.


In [5]:
CLIENT_ID = '135TWBFTCYNYOUUIHSHAD0KVAAVBCNQWGL0VMRVZOFGZ1DB5' # your Foursquare ID
CLIENT_SECRET = 'QQAA5TBAVT3FEOY1ELHKPSCIKBB5YO123FIJWLOANYPGZUM3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 135TWBFTCYNYOUUIHSHAD0KVAAVBCNQWGL0VMRVZOFGZ1DB5
CLIENT_SECRET:QQAA5TBAVT3FEOY1ELHKPSCIKBB5YO123FIJWLOANYPGZUM3


### Get Cities Data

First we need to retrieve a list of 100 pizzerias in the following cities (100 is th maximum that can be retrieved). To get only pizza resturants we need to set the venue category type to 'Pizza Place'. The venue category types can be found [here](https://developer.foursquare.com/docs/build-with-foursquare/categories/)

In [24]:
LIMIT = 100 
cities = ["New York, NY", 'Chicago, IL', 'New Haven, CT', 'San Francisco, CA', 'Orlando, FL']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [26]:
df_city={}
for city in cities:
    venues = results[city]['response']['groups'][0]['items']

    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name','venue.location.lat', 'venue.location.lng', 'venue.location.city', 'venue.rating']
    df_city[city] =nearby_venues.loc[:, filtered_columns]

    # clean columns
    df_city[city].columns = [col.split(".")[-1] for col in df_city[city].columns]

    print(df_city[city].head())

                  name        lat        lng      city  rating
0  Prince Street Pizza  40.723093 -73.994527  New York     NaN
1          Joe's Pizza  40.733234 -73.987672  New York     NaN
2       Champion Pizza  40.721638 -73.997470  New York     NaN
3                Posto  40.734737 -73.983049  New York     NaN
4               B Side  40.763986 -73.988145  New York     NaN
                            name        lat        lng     city  rating
0         Spacca Napoli Pizzeria  41.963325 -87.673739  Chicago     NaN
1                       Coalfire  41.891000 -87.659751  Chicago     NaN
2     Piece Brewery and Pizzeria  41.910403 -87.676145  Chicago     NaN
3  Candlelite Chicago Restaurant  42.017512 -87.690353  Chicago     NaN
4                      Homeslice  41.921867 -87.652453  Chicago     NaN
                             name        lat        lng       city  rating
0  Frank Pepe Pizzeria Napoletana  41.302991 -72.916962  New Haven     NaN
1                   Modern Apizza  41.31

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [23]:
for city in cities:
    print('{} venues were returned by Foursquare for {}.'.format(df_city[city].shape[0], city))

100 venues were returned by Foursquare for New York, NY.
100 venues were returned by Foursquare for Chicago, IL.
69 venues were returned by Foursquare for New Haven, CT.
100 venues were returned by Foursquare for San Francisco, CA.
100 venues were returned by Foursquare for Orlando, FL.


## Results

In [54]:
usa_lat = 35
usa_lng = -102
map_us = folium.Map(location=[usa_lat, usa_lng], zoom_start=4)
colors = ['blue', 'red', 'green','yellow', 'purple']

for index, city in enumerate(cities):

    # add markers to map
    for lat, lng, label in zip(df_city[city]['lat'], df_city[city]['lng'], df_city[city]['name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=colors[index],
            fill=True,
            fill_opacity=0.7,
            parse_html=False).add_to(map_us) 
  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")
map_us

Total number of pizza places in New York, NY =  254
Showing Top 100
Total number of pizza places in Chicago, IL =  216
Showing Top 100
Total number of pizza places in New Haven, CT =  91
Showing Top 100
Total number of pizza places in San Francisco, CA =  168
Showing Top 100
Total number of pizza places in Orlando, FL =  169
Showing Top 100


### Lets take a closer look at each city

In [67]:
colors = ['blue', 'red', 'green','yellow', 'purple']

maps = {}
for index, city in enumerate(cities):

    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    
    # add markers to map
    for lat, lng, label in zip(df_city[city]['lat'], df_city[city]['lng'], df_city[city]['name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=colors[index],
            fill=True,
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city]) 
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in New York, NY =  254
Showing Top 100
Total number of pizza places in Chicago, IL =  216
Showing Top 100
Total number of pizza places in New Haven, CT =  91
Showing Top 100
Total number of pizza places in San Francisco, CA =  168
Showing Top 100
Total number of pizza places in Orlando, FL =  169
Showing Top 100


In [59]:
maps['New York, NY']


In [63]:
maps['Chicago, IL']

In [64]:
maps['New Haven, CT']

In [65]:
maps['San Francisco, CA']

In [66]:
maps['Orlando, FL']

It seems that **New York, NY** has the largest number of pizzerias. Let's calculate the **density of restaurants in relation to the population** and try to extract some meaningfull info from that. 

### Density of restaurants in relation to the population

In [107]:
ny_pop = 8.399
ny_rest = results['New York, NY']['response']['totalResults']

ny_den = ny_rest/ny_pop
ny_den

30.241695439933327

In [106]:
ch_pop = 2.706
ch_rest = results['Chicago, IL']['response']['totalResults']

ch_den = ch_rest/ch_pop
ch_den

79.82261640798227

In [101]:
nh_pop = 0.130
nh_rest = results['New Haven, CT']['response']['totalResults']

nh_den = nh_rest/nh_pop
nh_den

700.0

In [102]:
sf_pop = 0.883
sf_rest = results['San Francisco, CA']['response']['totalResults']

sf_den = sf_rest/sf_pop
sf_den

190.26047565118913

In [103]:
ol_pop = 0.285
ol_rest = results['Orlando, FL']['response']['totalResults']

ol_den = ol_rest/ol_pop
ol_den

592.9824561403509

Based off of the **density of restaurants** in relation to the **population** New Haven, CT is the most dense city.   

### Density of restaurants in relation to the area

In [108]:
ny_area = 783.8
ny_rest = results['New York, NY']['response']['totalResults']

ny_den = ny_rest/ny_area
ny_den

0.32406226078081146

In [109]:
ch_area = 606.1
ch_rest = results['Chicago, IL']['response']['totalResults']

ch_den = ch_rest/ch_area
ch_den

0.3563768355056921

In [110]:
nh_area = 52.1
nh_rest = results['New Haven, CT']['response']['totalResults']

nh_den = nh_rest/nh_area
nh_den

1.746641074856046

In [111]:
sf_area = 121.4
sf_rest = results['San Francisco, CA']['response']['totalResults']

sf_den = sf_rest/sf_area
sf_den

1.3838550247116967

In [112]:
ol_area = 294.6
ol_rest = results['Orlando, FL']['response']['totalResults']

ol_den = ol_rest/ol_area
ol_den

0.5736591989137814

Based off of the **density of restaurants** in relation to the **area** New Haven, CT is still the most dense city.

## Discussion

One thing I noticed in the figure is that although New Haven has the smallest number of pizzerias it has the highest restaurnt density. 
So if you're looking for a place that pizza places are highly abundant and tightly packed then New Haven is the place for you. 
However, if you are looking a place with the most options then New York is the place for you.  

## Conclusion

Depending on what you are looking for New York and New Haven are the best options for Pizza Places 